# distiluse-base-multilingual-cased-v2 μοντέλο χωρίς tokenization. Υπολογισμός threshold βάσει των τυπικών αποκλίσεων (standard deviations) από τον μέσο όρο των cosine similarities και του μέσου όρου των similarities. Στην πρώτη περίπτωση 'προκύπτουν' 19 πιθανά hallucinations απο τα οποία το 50% είναι σωστά. Στη δεύτερη προκύπτουν 26 πιθανά hallucinations εκ των οποίων το 31.25% είναι σωστό (γνωρίζοντας ότι τα p(Hallucination) 32 datapoints ισούται με 1).

Θεωρώντας hallucinations τα datapoint με p(Hallucination) >= 0.66 Στην πρώτη περίπτωση 'προκύπτουν' 26 πιθανά hallucinations απο τα οποία το 50% είναι σωστά. Στη δεύτερη προκύπτουν 19 πιθανά hallucinations εκ των οποίων το 35.71% είναι σωστό

Για το αρχείο processed_df η πρώτη περίπτωση είχε 53.125% accuracy με 27 hallucinations και η δεύτερη 37.5% accuracy με 18 hallucinations για p(Hallucination)=1.

Για p(Hallucination)>=0.66 η πρώτη περίπτωση είχε 52.38% accuracy με 27 hallucinations και η δεύτερη 40.47% accuracy με 18 hallucinations.


Με το μοντέλο bert-base-multilingual-cased στο αρχείο trial-v1 και για p(Hallucination) = 1 έχω 32/32 με 59.375% **BEST SCORE**

In [ ]:
pip install -U sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4b653825fe272a2f8e1b0006ae3ede260783a566020fc095c5fac57dc592c51e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import json
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
model  = SentenceTransformer('bert-base-multilingual-cased')
#model  = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [ ]:
with open("trial-v1.json", "r") as f:
    trial_data = json.load(f)


df = pd.json_normalize(trial_data)

#df.head(79)

In [ ]:
result_dict = {}
hallucination_cosines_total = 0
hallucinations = []
hallucinations_cosines=[]
cosine_similarities = []

for i in range(0, len(df)):
    sentences = [df.iloc[i]['hyp'], df.iloc[i]['tgt']]
    embedding = model.encode(sentences, convert_to_tensor=False) #tokenization is implicitly handled by the SentenceTransformer here
    cosine_scores = util.cos_sim(embedding, embedding)

    for id_h, h in enumerate(sentences):
        for id_t, t in enumerate(sentences):
            if id_h >= id_t:
                continue
            dp_id = len(result_dict)  #Using dp_id as the key
            #if df.iloc[i]['p(Hallucination)'] >= 0.66:
            if df.iloc[i]['p(Hallucination)'] == 1:
                hallucinations.append(dp_id)
                hallucination_cosines_total += cosine_scores[id_h][id_t].item()
                hallucinations_cosines.append(cosine_scores[id_h][id_t].item())

            result_dict[dp_id] = {'hyp': h, 'tgt': t, 'cosine_similarity': cosine_scores[id_h][id_t].item()}

for dp_id, values in result_dict.items():
    cosine_similarities.append(values['cosine_similarity'])
    print(f'Datapoint {dp_id}:')
    print(f'hyp: {values["hyp"]}')
    print(f'tgt: {values["tgt"]}')
    print(f'cosine similarity: {values["cosine_similarity"]}')
    print()


Datapoint 0:
hyp: A district of Kowloon, China.
tgt: The Chaoshan region where the Teochew dialect is spoken.
cosine similarity: 0.5546447038650513

Datapoint 1:
hyp: (Wicca) A witchdoctor.
tgt: (South Africa) A traditional tribal diviner or spiritual healer.
cosine similarity: 0.6523058414459229

Datapoint 2:
hyp: (nautical) A halyard.
tgt: (nautical) A staff that carries a flag or pennant above the mast of a sailboat.
cosine similarity: 0.5773873329162598

Datapoint 3:
hyp: The quality of being rational.
tgt: (archaic) The quality or state of being rational.
cosine similarity: 0.7895265221595764

Datapoint 4:
hyp: (uncountable) The study of trees.
tgt: The worship of trees.
cosine similarity: 0.5450116395950317

Datapoint 5:
hyp: (zoology) A pseudostome.
tgt: (biology) The anal opening on the pseudembryo of an echinoderm.
cosine similarity: 0.6021667718887329

Datapoint 6:
hyp: Resembling or characteristic of umber.
tgt: Of or pertaining to umber; like umber.
cosine similarity: 0.697

In [ ]:
threshold = np.mean(hallucinations_cosines)

hallucinations_pred = []

for dp_id, values in result_dict.items():
    similarity = values['cosine_similarity']
    if similarity < threshold:
        hallucinations_pred.append(dp_id)

counter1 = Counter(hallucinations)
counter2 = Counter(hallucinations_pred)

intersection = counter1 & counter2

total_mutual_elements = sum(intersection.values())

print(f'Threshold: {threshold}')
print(f'Total Hallucinations: {len(hallucinations)}')
print(f'Total Hallucinations Predicted: {len(hallucinations_pred)}')
print('Percentage of correctly predicted hallucinations:' , (total_mutual_elements / len(hallucinations)) * 100 , '%')
print('Correct Hallucinations:', intersection.keys())
print('Number of correct Hallucinations:', len(intersection.keys()))
print(hallucinations)

Threshold: 0.6741333743557334
Total Hallucinations: 32
Total Hallucinations Predicted: 32
Percentage of correctly predicted hallucinations: 59.375 %
Correct Hallucinations: dict_keys([0, 4, 5, 7, 9, 15, 19, 23, 25, 30, 33, 34, 37, 40, 48, 60, 65, 67, 79])
Number of correct Hallucinations: 19
[0, 4, 5, 7, 9, 12, 14, 15, 19, 23, 25, 30, 31, 33, 34, 35, 37, 40, 41, 48, 52, 57, 60, 63, 64, 65, 67, 69, 70, 74, 78, 79]


In [ ]:
mean_similarity = np.mean(cosine_similarities)
std_deviation = np.std(cosine_similarities)

threshold_in_std_devs = 0.9
threshold = mean_similarity - (threshold_in_std_devs * std_deviation)

potential_hallucinations = [i for i, similarity in enumerate(cosine_similarities) if similarity < threshold]

counter1 = Counter(hallucinations)
counter2 = Counter(potential_hallucinations)

intersection = counter1 & counter2

total_mutual_elements = sum(intersection.values())

print(f'Threshold: {threshold}')
print(f'Total Hallucinations: {len(hallucinations)}')
print(f'Total Hallucinations Predicted: {len(potential_hallucinations)}')
print('Percentage of correctly predicted hallucinations:' , (total_mutual_elements / len(hallucinations)) * 100 , '%')
print('Correct Hallucinations:', intersection.keys())


Threshold: 0.5988328488446315
Total Hallucinations: 32
Total Hallucinations Predicted: 17
Percentage of correctly predicted hallucinations: 31.25 %
Correct Hallucinations: dict_keys([0, 4, 9, 15, 19, 23, 25, 33, 37, 60])
